# Part1: Collecting Data from Skeleton Info

Import related packages

In [1]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
#!pip install python_highcharts
from highcharts import Highchart
import json

Using TensorFlow backend.


Function Introduction: 

    1: getJsonList: given a folder path, read file names from this folder and return a list
    
    2: getJsonForOneFrame: given a json file name, which is the information from a frame, return one person's information with 190 data points as a list describing face and body
    
    3: Processing one json file for a frame, if there are multiple people, we use this function to select the person with the most data points 

# Part5 Testing on new video and generate outcomes

In [22]:
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")
model = load_model('final.h5')
model.summary()
score1 = model.evaluate(x_test, y_test)
score2 = model.evaluate(x_train, y_train)
print("Accuracy on test data: ", score1[1])
print("Accuracy on train data: ", score2[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 30, 1024)          2879488   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
dense_layer (Dense)          (None, 256)               262400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 1)                 257       
Total params: 9,437,697
Trainable params: 9,437,697
Non-trainable params: 0
_________________________________________________________________
10660/10660 [==============================] - 13s 1ms/step
Accuracy on test data:  0.7422622442245483
Accurac

In [3]:
def getJsonList(filePath):
    fileList = []
    for i,j,k in os.walk(filePath):
        fileList = k
    return fileList
def calcWeight(item):
    zeros = 0
    for i in item['pose_keypoints_2d']:
        if i == 0:
            zeros += 1
    for i in item['face_keypoints_2d']:
        if i == 0:
            zeros +=1
    return zeros

def getJsonForOneFrame(fileName, filePath):
    l = getJsonList(filePath)
    with open(fileName,'r') as f:
        temp = json.loads(f.read())
        allPeople = temp['people']
    if len(allPeople) == 0:
        empty = []
        for i in range(190):
            empty.append(0)
        return empty
    target = allPeople[0]
    for dic in allPeople:
        if calcWeight(dic) < calcWeight(target):
            target = dic
    matrix = []
    if(len(target['pose_keypoints_2d']) == 0):
        for i in range(50):
            matrix.append(0)
    else:
        for i in range(75):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['pose_keypoints_2d'][i])
    if(len(target['face_keypoints_2d']) == 0):
        for i in range(140):
            matrix.append(0)
    else:
        for i in range(210):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['face_keypoints_2d'][i])
    return matrix


def writeData(path, pathType,fullDfList):
    fileList = getJsonList(path)
    fileList = set(fileList)
    print(len(fileList))
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        temp = int(row['name'][:-1])
        if(row["type"] != pathType):
            pass
        else:
            startName = row["name"]+"_"+'0'.zfill(12)+"_keypoints.json"
            endName = row["name"]+"_"+'30'.zfill(12)+"_keypoints.json"
            if endName and endName not in fileList:
                print('Video ',row["name"]," is not long enough")
            else:
                initFrame = 0
                while(row["name"]+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
                    matrix = []
                    for i in range(30):
                        file = row["name"]+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
                        matrixLine = getJsonForOneFrame(path+'/'+file,path)
                        matrix.append(matrixLine)
                    if row['type'] == 'shortyes':
                        label = True
                    elif row['type'] == 'shortno':
                        label = False
                    else:
                        label = getLable(dataCat,row['name'], initFrame)
                    data = [[label, matrix]]
                    print("Data:",label, row['name'], initFrame,'matrix:',len(matrix),len(matrix[0]))
                    df = pd.DataFrame(data,columns=['lable','matrix'])
                    fullDfList.append(df)
                    initFrame += 30
    return fullDfList

def draw(folderName,videoName, modelName):
    #videoName = '6'#define your test video name here
    fileList = getJsonList(folderName)
    fileList = set(fileList)
    print(len(fileList))
    fullData = []
    initFrame = 0
    while(folderName+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
        matrix = []
        for i in range(30):
            file = folderName+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
            matrixLine = getJsonForOneFrame(folderName+'/'+file,folderName)

            matrix.append(matrixLine)
        fullData.append(matrix)
        initFrame += 30
    def cleanX(raw):
        length = len(raw)
        temp = np.zeros((length, 30,190))
        for i in range(len(raw)):
            for j in range(30):
                for k in range(190):
                    temp[i][j][k] = raw[i][j][k]
            print("progress:{0}%".format(round((i + 1) * 100 / len(raw))), end="\r")
        return temp
    temp = cleanX(fullData)
    model = load_model(modelName)
    predict = model.predict(temp)
    a = predict.tolist()
    result = []
    frames = 30
    for i in a:
        result.append([frames,i[0]])
        frames += 30
    #print(result)
    chart = Highchart()
    chart.set_options('chart', {'inverted': False})
    options = {
        'title': {
            'text': 'Prediction for video '+folderName+'.mp4'
        },
        'subtitle': {
            'text': '1 means talking while 0 means non-talking'
        },
         'xAxis': {
            'title': {
                'text': 'Second'
            }
        },
        'yAxis': {
            'title': {
                'text': 'Flag'
            },
        }
    }
    chart.set_dict_options(options)
    chart.add_data_set(result, series_type='line', name='prediction')
    
    
    
    index = []
    for i in result:
        for j in range(30):
            index.append(i[1])
            
        
    import cv2
    video = videoName
    result_video = folderName+'_result.mp4'
    cap = cv2.VideoCapture(video)
    fps_video = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    videoWriter = cv2.VideoWriter(result_video, fourcc, fps_video, (frame_width, frame_height))
    frame_id = 0
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame_id += 1
            left_x_up = int(frame_width / frame_id)
            left_y_up = int(frame_height / frame_id)
            right_x_down = int(left_x_up + frame_width / 10)
            right_y_down = int(left_y_up + frame_height / 10)
            word_x = left_x_up + 5
            word_y = left_y_up + 25
            if frame_id >= len(index):
                pass
            else:
                cv2.putText(frame, 'frame_%s' %index[frame_id], (word_x, word_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,255,155), 2)
            videoWriter.write(frame)
        else:
            videoWriter.release()
            break    
    
    frameRate = 30#default video frame rate
    for i in result:
        i[0] = i[0]/frameRate
    print(result)
    
    d = {}
    d['talking'] = result
    file_name = 'timeLabel_'+videoName+'.json' 
    with open(file_name,'w') as file_object:
        json.dump(d,file_object)
    return chart

In [4]:
folder_name = '1'#type your folder path here, which contains openpose json files
video_name = '1.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'final.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

705
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[[1.0, 0.8638039827346802], [2.0, 0.8766095638275146], [3.0, 0.8733783960342407], [4.0, 0.8832111358642578], [5.0, 0.9061653017997742], [6.0, 0.8972619771957397], [7.0, 0.873337984085083], [8.0, 0.8712981343269348], [9.0, 0.8779884576797485], [10.0, 0.8888360857963562], [11.0, 0.9068564772605896], [12.0, 0.90848708152771], [13.0, 0.9086014032363892], [14.0, 0.8716625571250916], [15.0, 0.8595634698867798], [16.0, 0.8736330270767212], [17.0, 0.8963184952735901], [18.0, 0.8871179819107056], [19.0, 0.88200443983078], [20.0, 0.8719789981842041], [21.0, 0.8419056534767151], [22.0, 0.8726983070373535], [23.0, 0.8735845685005188]]


In [5]:
folder_name = '2'#type your folder path here, which contains openpose json files
video_name = '2.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'final.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

2797
[[1.0, 0.756384015083313], [2.0, 0.7920012474060059], [3.0, 0.7920012474060059], [4.0, 0.7920011281967163], [5.0, 0.7455164790153503], [6.0, 0.7492484450340271], [7.0, 0.7123009562492371], [8.0, 0.7061437368392944], [9.0, 0.6958250999450684], [10.0, 0.7326527237892151], [11.0, 0.7504825592041016], [12.0, 0.7440016269683838], [13.0, 0.7581945657730103], [14.0, 0.7920012474060059], [15.0, 0.7894208431243896], [16.0, 0.8199425935745239], [17.0, 0.8558235168457031], [18.0, 0.8392744660377502], [19.0, 0.8365793228149414], [20.0, 0.8559845685958862], [21.0, 0.8616189360618591], [22.0, 0.8597147464752197], [23.0, 0.861438512802124], [24.0, 0.8588628768920898], [25.0, 0.8597329258918762], [26.0, 0.8561543822288513], [27.0, 0.861635684967041], [28.0, 0.861635684967041], [29.0, 0.861635684967041], [30.0, 0.861635684967041], [31.0, 0.861635684967041], [32.0, 0.861635684967041], [33.0, 0.8616355657577515], [34.0, 0.861635684967041], [35.0, 0.861632764339447], [36.0, 0.8616353273391724], [37.0

In [6]:
folder_name = '3'#type your folder path here, which contains openpose json files
video_name = '3.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'final.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

2392
[[1.0, 0.7515640258789062], [2.0, 0.7487304210662842], [3.0, 0.7602242231369019], [4.0, 0.7840092778205872], [5.0, 0.8338911533355713], [6.0, 0.8387223482131958], [7.0, 0.8475663661956787], [8.0, 0.8409662246704102], [9.0, 0.715920090675354], [10.0, 0.3984103798866272], [11.0, 0.3426392376422882], [12.0, 0.33530208468437195], [13.0, 0.37121960520744324], [14.0, 0.5008561611175537], [15.0, 0.6102865934371948], [16.0, 0.762194812297821], [17.0, 0.5306972861289978], [18.0, 0.5971373915672302], [19.0, 0.8262979984283447], [20.0, 0.884749174118042], [21.0, 0.9086089730262756], [22.0, 0.9086100459098816], [23.0, 0.8935744166374207], [24.0, 0.8626834154129028], [25.0, 0.8668770790100098], [26.0, 0.8695667386054993], [27.0, 0.8214773535728455], [28.0, 0.4461328983306885], [29.0, 0.4562254846096039], [30.0, 0.8064184188842773], [31.0, 0.8380129933357239], [32.0, 0.8573476672172546], [33.0, 0.7863193154335022], [34.0, 0.5999335050582886], [35.0, 0.5714821219444275], [36.0, 0.703888893127441

In [7]:
folder_name = '4'#type your folder path here, which contains openpose json files
video_name = '4.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'final.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

2980
[[1.0, 0.6035168766975403], [2.0, 0.5987865328788757], [3.0, 0.5842889547348022], [4.0, 0.6123222708702087], [5.0, 0.5967644453048706], [6.0, 0.5837297439575195], [7.0, 0.5837297439575195], [8.0, 0.5776056051254272], [9.0, 0.5837275981903076], [10.0, 0.5286592841148376], [11.0, 0.5487819314002991], [12.0, 0.5243111848831177], [13.0, 0.4980601370334625], [14.0, 0.5420829653739929], [15.0, 0.5218074321746826], [16.0, 0.5466016530990601], [17.0, 0.5663735270500183], [18.0, 0.5837297439575195], [19.0, 0.5837333798408508], [20.0, 0.5837347507476807], [21.0, 0.5837297439575195], [22.0, 0.5837297439575195], [23.0, 0.5837101936340332], [24.0, 0.5296322107315063], [25.0, 0.4977944493293762], [26.0, 0.49782225489616394], [27.0, 0.4977940618991852], [28.0, 0.49778687953948975], [29.0, 0.49779048562049866], [30.0, 0.5010179281234741], [31.0, 0.4977872967720032], [32.0, 0.49778077006340027], [33.0, 0.4977896213531494], [34.0, 0.49779003858566284], [35.0, 0.497790664434433], [36.0, 0.5350268483

In [8]:
folder_name = '5'#type your folder path here, which contains openpose json files
video_name = '5.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'final.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1816
[[1.0, 0.026777684688568115], [2.0, 0.02484115958213806], [3.0, 0.011576861143112183], [4.0, 0.006020694971084595], [5.0, 0.007784545421600342], [6.0, 0.03992530703544617], [7.0, 0.06753212213516235], [8.0, 0.06522685289382935], [9.0, 0.0858817994594574], [10.0, 0.06477612257003784], [11.0, 0.06831875443458557], [12.0, 0.06485912203788757], [13.0, 0.06813359260559082], [14.0, 0.0800815224647522], [15.0, 0.0770384669303894], [16.0, 0.033513009548187256], [17.0, 0.42926251888275146], [18.0, 0.7665005922317505], [19.0, 0.1741722822189331], [20.0, 0.06024286150932312], [21.0, 0.22291702032089233], [22.0, 0.2527516186237335], [23.0, 0.18900829553604126], [24.0, 0.170061856508255], [25.0, 0.07622891664505005], [26.0, 0.04960915446281433], [27.0, 0.06166231632232666], [28.0, 0.06465032696723938], [29.0, 0.09586715698242188], [30.0, 0.09529420733451843], [31.0, 0.08412644267082214], [32.0, 0.1371784806251526], [33.0, 0.039201050996780396], [34.0, 0.5358226299285889], [35.0, 0.315269231796